# Restaurant Visitor Forecasting

### 1. Data Preprocessing

In [4]:
import pandas as pd

air_visit = pd.read_csv('data/air_visit_data.csv')
air_reserve = pd.read_csv('data/air_reserve.csv')
hpg_reserve = pd.read_csv('data/hpg_reserve.csv')
air_store_info = pd.read_csv('data/air_store_info.csv')
hpg_store_info = pd.read_csv('data/hpg_store_info.csv')
store_id_relation = pd.read_csv('data/store_id_relation.csv')
date_info = pd.read_csv('data/date_info.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')

Converting to datetime format

In [14]:
for df in [air_reserve, hpg_reserve]:
    df['visit_datetime'] = pd.to_datetime(df['visit_datetime']).dt.date
    df['reserve_datetime'] = pd.to_datetime(df['reserve_datetime']).dt.date

air_visit['visit_date'] = pd.to_datetime(air_visit['visit_date']).dt.date
date_info['calendar_date'] = pd.to_datetime(date_info['calendar_date']).dt.date

Aggregating Reservation Data
- Total reservations per day
- Average reservation lead time (gap between reservation and visit)

In [15]:
air_reserve['reserve_lead_time'] = (
    pd.to_datetime(air_reserve['visit_datetime']) - 
    pd.to_datetime(air_reserve['reserve_datetime'])
).dt.days

air_agg = air_reserve.groupby(['air_store_id', 'visit_datetime']).agg({
    'reserve_visitors': 'sum',
    'reserve_lead_time': 'mean'
}).reset_index().rename(columns={
    'visit_datetime': 'visit_date',
    'reserve_visitors': 'air_reserve_visitors',
    'reserve_lead_time': 'air_reserve_lead_time'
})

In [16]:
hpg_reserve = hpg_reserve.merge(store_id_relation, on='hpg_store_id', how='inner')

hpg_reserve['reserve_lead_time'] = (
    pd.to_datetime(hpg_reserve['visit_datetime']) - 
    pd.to_datetime(hpg_reserve['reserve_datetime'])
).dt.days

hpg_agg = hpg_reserve.groupby(['air_store_id', 'visit_datetime']).agg({
    'reserve_visitors': 'sum',
    'reserve_lead_time': 'mean'
}).reset_index().rename(columns={
    'visit_datetime': 'visit_date',
    'reserve_visitors': 'hpg_reserve_visitors',
    'reserve_lead_time': 'hpg_reserve_lead_time'
})

Creating the modeling base table

In [17]:
# Merge visitor data with reservation features
full_df = air_visit.merge(air_agg, on=['air_store_id', 'visit_date'], how='left')
full_df = full_df.merge(hpg_agg, on=['air_store_id', 'visit_date'], how='left')

# Merge with restaurant metadata
full_df = full_df.merge(air_store_info, on='air_store_id', how='left')

# Merge with calendar features
date_info = date_info.rename(columns={'calendar_date': 'visit_date'})
full_df = full_df.merge(date_info, on='visit_date', how='left')

# Add weekday and weekend features
full_df['day_of_week'] = pd.to_datetime(full_df['visit_date']).dt.dayofweek
full_df['is_weekend'] = full_df['day_of_week'].isin([5, 6]).astype(int)

In [18]:
full_df.head()

,air_store_id,visit_date,visitors,air_reserve_visitors,air_reserve_lead_time,hpg_reserve_visitors,hpg_reserve_lead_time,air_genre_name,air_area_name,latitude,longitude,day_of_week,holiday_flg,is_weekend
0,air_ba937bf13d40fb24,2016-01-13,25,NaN,NaN,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,2,0,0
1,air_ba937bf13d40fb24,2016-01-14,32,NaN,NaN,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,3,0,0
2,air_ba937bf13d40fb24,2016-01-15,29,NaN,NaN,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,4,0,0
3,air_ba937bf13d40fb24,2016-01-16,22,NaN,NaN,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,5,0,1
4,air_ba937bf13d40fb24,2016-01-18,6,NaN,NaN,NaN,NaN,Dining bar,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,0,0,0


### 2. Lag + Rolling Features